# Federated learning: regression using the California Housing database

In this notebook we are going to show how you can use a federated learning environment to create a regression model. 
In the notebook on the [Linear regression for a 2D simple case](./federated_models_regression_linear.ipynb) we explained the basic concepts of the framework, so now we will go slightly faster.

First of all, we load a dataset (included in the framework) to allow for regression experiments.

In [ ]:
import shfl
from shfl.data_base.california_housing import CaliforniaHousing

database = CaliforniaHousing()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

We are going to explore the data

In [ ]:
print("Shape of train_data: " + str(train_data.shape))
print("Shape of train_labels: " + str(train_labels.shape))
print("One sample features: " + str(train_data[0]))
print("One sample label: " + str(train_labels[0]))

Federeted data generation:

In [ ]:
import shfl

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_label = iid_distribution.get_federated_data(20, percent=10)

Model definition:

In [ ]:
import keras

def model_builder():
    # create model
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='normal'))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=["mae"])
    
    return shfl.model.DeepLearningModel(model)

Federated environment definition:

In [ ]:
aggregator = shfl.federated_aggregator.AvgFedAggregator()
federated_government = shfl.learning_approach.FederatedGovernment(model_builder, federated_data, aggregator)

Reshaping data:

In [ ]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.label = np.reshape(labeled_data.label, (labeled_data.label.shape[0], 1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

Running experiment:

In [ ]:
test_label = np.reshape(test_label, (test_label.shape[0], 1))
federated_government.run_rounds(3, test_data, test_label)